<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
%%writefile get_data.py
from pathlib import Path
import requests
import os
from zipfile import ZipFile
import argparse

# setup parser
parser = argparse.ArgumentParser(description="get download link")

# get link
parser.add_argument(
    "--url",
    default="",
    type=str,
    help="link to image repository"
)

# get arguments from parser
args = parser.parse_args()

# get url
url = args.url


### download data from url ###
def download_data(source: str,
                 destination: str="images",
                 remove_source: bool=True,
                 INFO: bool=True):
    
    """Download file from an url to data, extract files to images folder
    Args:
        source: url to the data source
    Returns:
        image_path: folder to store train and test images"""

    # setup data and image paths
    data_path = Path("data")
    image_path = data_path / destination

    # check if image_path exist and create the dir
    if image_path.is_dir() and len(os.listdir(image_path)) > 0 :
        if INFO:
            print(f"[INFO] {image_path} exist.")
    else:
        if INFO:
            print(f"[INFO] creating {image_path}...")
        image_path.mkdir(parents=True, exist_ok=True)

        zip_file_name = Path(source).name
        zip_file_path = data_path / zip_file_name
        if INFO:
            print(f"[INFO] downloading {zip_file_name} to {data_path}...")
        with open(zip_file_path, "wb") as f:
            request = requests.get(url=source)
            f.write(request.content)
        if INFO:
            print(f"[INFO] extracting {zip_file_name} to {image_path}...")
        with ZipFile(zip_file_path) as zip_ref:
            zip_ref.extractall(image_path)
    
        if remove_source:
            if INFO:
                print(f"[INFO] removing {zip_file_name}...")
            os.remove(zip_file_path)

    return image_path


download_data(source=url)

Writing get_data.py


In [2]:
# Example running of get_data.py
!python get_data.py --url "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"

[INFO] creating data/images...
[INFO] downloading pizza_steak_sushi.zip to data...
[INFO] extracting pizza_steak_sushi.zip to data/images...
[INFO] removing pizza_steak_sushi.zip...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [3]:
%%writefile train.py

# YOUR CODE HERE
import torch
import torchvision
from torch import nn
from torchvision import transforms
import argparse

from sources import models, engine, utils, datasetup

# setup parser
parser = argparse.ArgumentParser(description="get training parameters")

# add train dir
parser.add_argument(
    "--train_dir",
    type=str,
    default="data/images/train/",
    help="training data"
)

# add test dir
parser.add_argument(
    "--test_dir",
    type=str,
    default="data/images/test/",
    help="testing data"
)

# add dataloader batch size
parser.add_argument(
    "--batch_size",
    type=int,
    default=32,
    help="dataloader batch size"
)

# add number of worker for dataloader
parser.add_argument(
    "--num_workers",
    type=int,
    default=1,
    help="number of worker for dataloader"
)

# add number of epochs for training
parser.add_argument(
    "--epochs",
    type=int,
    default=2,
    help="number of epochs for training"
)

# add the learning rate
parser.add_argument(
    "--lr",
    type=float,
    default=1e-3,
    help="learning rate"
)

# get arguments from parser
args = parser.parse_args()

# get parameters
train_dir = args.train_dir
test_dir = args.test_dir
batch_size = args.batch_size
num_workers = args.num_workers
epochs = args.epochs
lr = args.lr


# device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"

# setup pretrained efficient net model
model, model_transforms = models.create_effnet(
    effnet_version=0,
    num_class_names=3,
    device=device
)

# add augmented methods to train transformation
train_transforms = transforms.Compose([
    transforms.TrivialAugmentWide(),
    model_transforms
])

# setup dataloaders
train_dataloader, test_dataloader, class_names = datasetup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transforms=train_transforms,
    test_transforms=model_transforms,
    batch_size=batch_size,
    num_workers=num_workers
)

# setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

# do training
results = engine.train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=epochs,
    device=device
)

saved_model_path = utils.save_model(
    model=model,
    model_dir="saved_model",
    model_name_grid=[model.name]
)


Writing train.py


In [4]:
!python train.py --epochs 10 --batch_size 16

2023-11-28 14:59:10.323640: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 14:59:10.361069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 14:59:10.961760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[INFO] creating EfficientNet_B0...
[INFO] creating dataloaders... 
train_dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7f7af783da50> 
test_dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7f7af783ead0> 
number of clas

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [5]:
%%writefile predict.py

# YOUR CODE HERE
import torch
from pathlib import Path
from PIL import Image
import random
import argparse

from sources import models

# setup parser
parser = argparse.ArgumentParser(description="get path to pimage")
# add path to arg
parser.add_argument(
    "--img_path",
    type=str,
    default=".",
    help="path to an image for label prediction"
)
# get arg parameters
args = parser.parse_args()
# get image path
img_path = args.img_path


# # image path
# img_path = random.choice(list(Path("data/images/").glob("*/*/*")))
true_label = Path(img_path).parent.stem

# device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"

# setup pretrained efficient net model
loaded_model, loaded_model_transforms = models.create_effnet(
    effnet_version=0,
    num_class_names=3,
    device=device
)

# class names
class_names = ['pizza', 'steak', 'sushi']

# load state dict
loaded_model.load_state_dict(torch.load("saved_model/EfficientNet_B0.pth"))

# open image using PIL
img = Image.open(img_path)

# transform and send image to device
img_transformed = loaded_model_transforms(img).unsqueeze(dim=0).to(device)

# switch to evaluation mode
loaded_model.eval()
with torch.inference_mode():
    loaded_y_logit = loaded_model(img_transformed)
    loaded_y_preds = torch.argmax(torch.softmax(loaded_y_logit, dim=1), dim=1)
    print(f"Prediction: {class_names[loaded_y_preds]} | true label: {true_label}")

Writing predict.py


In [6]:
!python predict.py --img_path "data/images/test/pizza/1152100.jpg"

[INFO] creating EfficientNet_B0...
Prediction: pizza | true label: pizza
